# 使用Python批量下载哨兵一号（sentinel-1）的精密轨道数据

## 1. 安装Python及data_downloader包

本文使用的是Python脚本，来批量下载文件。如果使用本文的下载方式，请先下载Python3.6以上的版本，并使用pip安装data_downloader包:

```bash
pip install data_downloader
```

> **注意：** 为了方便使用，现已将sentinel-1 orbit数据集成到了data_downloader。如果之前装过data_downloader, 执行下面脚本前，需要升级你的版本（version>=0.5.1）：
```bash
pip install data_downloader -U
```

## 2. 将登录信息写入 .netrc 文件

本教程使用从AFS下载精密轨道数据需要NASA的授权，所以需要在`.netrc`中加入登录信息，这样下载的时候就不会因登录信息而被拒绝访问。

将下面代码中的`your_username`与`your_password`，改为自己在哨兵官网注册的用户名与密码，并复制到Python编辑器中执行。

>**注意：** 如果.netrc文件已经有NASA的登录信息，这步可以跳过

In [ ]:
from data_downloader import downloader, parse_urls
from pathlib import Path
import pandas as pd
import datetime as dt

netrc = downloader.Netrc()
netrc.add("urs.earthdata.nasa.gov", "your_username", "your_password")

# 查看写入结果
print(netrc)

## 2. 下载精密轨道数据与辅助数据
哨兵一号的精密轨道数据（aux_poeorb, Precise orbit）与辅助数据(aux_cal,  Auxiliary data) 可以从欧空局与AFS下载。由于欧空局这两年换了几次下载链接，本教程采用的是从AFS上下载。

复制并运行下面代码即可下载。

> Tip： data_downloader支持断点续传，如果程序意外中断，重新执行脚本即可，程序会跳过下载完成的文件，并继续下载未下载完全的部分。

In [ ]:
from data_downloader import downloader, SentinelOrbit
from pathlib import Path

folder_cal = Path("/media/data/aux_cal")  # 指定aux_cal，**辅助数据** 的下载文件夹
folder_preorb = Path("/media/data/poeorb")  # 指定aux_poeorb，**精密轨道数据** 的下载文件夹

s1_orbit = SentinelOrbit()

############# download aux_cal  #####################
urls_cal = s1_orbit.cal_urls(platform="all")  # 获取S1A与S1B的所有辅助数据的链接
downloader.async_download_datas(urls_cal, folder=folder_cal) # 执行下载


########### download precise orbit #####################
urls_preorb = s1_orbit.poeorb_urls(
    date_start="20210101", date_end="20220301", platform="S1A"
)  # 获取所有S1A在20210101-20220301期间的精密轨道数据的链接
downloader.download_datas(urls_preorb, folder=folder_preorb) # 执行下载